# Pruebas una red neuronal con los datos de test
En este notebook se  prueba una red neuronal con los datos descritos en el paper de [Haoming Zhang
et al 2021](https://iopscience.iop.org/article/10.1088/1741-2552/ac2bf8/meta). La red neuronal es feed forward y se entrena con datos sintéticos.

In [9]:
# cargamos las librerias necesarias
import numpy as np
import math
import mne
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from keras import layers
from keras import models


In [10]:
# cargamos los datos limpios que estan en el archivo EEG_all_epochs.npy de la carpeta data
EEG_clean=np.load('data/EEG_all_epochs.npy')
# cargamos los datos de artefactos oculares EOG que estan en el archivo EOG_all_epochs.npy de la carpeta data y los almacenas en una variable EEG_EOG
EEG_EOG=np.load('data/EOG_all_epochs.npy')


### Creación de la señal EEG contaminada
Para crear la señal contaminada se realiza la operación $y=x+\lambda \cdot n$. Donde $x$ es la señal original, $n$ es el ruido y $\lambda$ es el factor de contaminación. Lambda se calcula a partir de la relación señal a ruido deseada, mediante la fórmula:
$$
SNR=10\log_{10}\left(\frac{RMS(EEG_{limpio})}{RMS(ruido)}\right)
$$
Donde:
$$
RMS(EEG_{limpio})=\sqrt{\frac{1}{N}\sum_{i=1}^{N}x_{i}^{2}}
$$
corresponde a la raiz cuadratica media de la señal limpia y la raiz cuadrática media del ruido introducido es:
$$
RMS(ruido)=\sqrt{\frac{1}{N}\sum_{i=1}^N(\lambda\cdot n_i)^2}
$$


In [3]:
x=EEG_clean.copy()
n=EEG_EOG.copy()
#Se toman la misma cantidad de muestras de señal limpia y ruido
x=x[0:n.shape[0],:]
# calculo de rms de la señal limpia
rms_sample = np.sqrt(np.mean(x**2, axis=1))
rms_general_x=np.mean(rms_sample)
# calculo de rms del ruido
rms_sample_n = np.sqrt(np.mean(n**2, axis=1))
rms_general_n=np.mean(rms_sample_n)


In [6]:
print(rms_sample)

218.9780209223996 128.67397466480674


### Calculo de lambda
Para calcular lambda se utiliza la siguiente expresión:
$$
\lambda = \sqrt{\frac{\sum_{i=1}^{N}x_i^2}{10^{\frac{2SNR}{10}}\sum_{i=1}^{N}n_i^2}}
$$

In [8]:
# lamba es igual a rms_x dividido rms_n
lambd = rms_general_x/rms_general_n


In [ ]:


def generar_seno():
    # Definir el período de la señal
    periodo = 32
    # Crear el vector de tiempo
    t = np.linspace(0, periodo, 256, endpoint=False)
    # Generar la señal seno
    seno = np.sin(2 * np.pi * t / periodo)
    # Normalizar la amplitud a 1
    seno = seno / np.max(np.abs(seno))
    return seno